In [1]:
import numpy as np
import h5py

filename = "camera_intrinsics.h5"
filenameEx = "extrinsics.h5"
main_K, main_distortion = None, None
with h5py.File(filename, "r") as data:

    main_K = np.array(data["main"]["K"])
    main_distortion = np.array(data["main"]["distortion"])

#get the main camera extrinsics

with h5py.File(filenameEx,"r") as data:
    main_EX = np.array(data["main"])
dist_pickle = {}
k = main_K
dst = main_distortion
R_ex = np.zeros((3, 3))

for i in range(0,3):
    for k in range(0,3):
        R_ex[i][k] = main_EX[i][k]

[[-0.0745772   0.01507948  0.99710125  2.0568478 ]
 [-0.99657786  0.03461772 -0.07506159 -0.05882518]
 [-0.03564926 -0.99928683  0.01244618  1.4660132 ]
 [ 0.          0.          0.          1.        ]]
[[-0.0745772   0.01507948  0.99710125  2.0568478 ]
 [-0.99657786  0.03461772 -0.07506159 -0.05882518]
 [-0.03564926 -0.99928683  0.01244618  1.4660132 ]
 [ 0.          0.          0.          1.        ]]


In [4]:
def uvtoXYZ(u, v, main_k):
    image_vec = np.array([u, v, 1]).T # 2D points in image
    n = np.array([0,0,1]).T  #or is it [0,1,0]? .. normal vector in car/road frame
    cam_ex = main_EX # extrinsic matrix for main cam
    h = 1.4660132 # height of main cam (z tranlastion in extrinsic)
    
    n_c = np.matmul(R_ex, n) #normal vector in cam frame
    print(n_c) #ok makes sense
    V = h/(np.matmul(n_c.T ,np.matmul(np.linalg.inv(main_K),image_vec.T)))
    print(V)
    pts_in_camera_frame = V * np.matmul(np.linalg.inv(main_K),image_vec)
    return pts_in_camera_frame


test = np.zeros(4)
y = uvtoXYZ(1, 50, main_K)
#print(y)
#testing
test[0] = y[0]
test[1] = y[1]
test[2] = y[2]
test[3] = 1
print(np.matmul(np.linalg.inv(R_ex),y))

[ 0.99710125 -0.07506159  0.01244618]
-4.027923644388297
[-0.92005675  4.08188032  1.46601313]
